 Metropolis–Hastings for Taxi Matchings (CMOR 451 Project)
 
 Metropolis–Hastings (MH) simulation where:
 
 - **States** are matchings between riders and drivers.
 - **Utility** of a matching is:  $$U(M) = \log \mathbb{P}\big(\max_i T_i(M) \le \text{time\_threshold}\big),$$  where $T_i(M)$ is the total time (pickup + trip) for rider $i$.
 
 - Pickup and trip times are modeled as **lognormal** random variables, so total time is a **sum of lognormals** and we approximate the CDF via Monte Carlo.
 - The **target distribution** over matchings is  $$\pi(M) \propto e^{U(M)}.$$
 
 - We also compute **CVaR** of the mean rider time as a risk / dissatisfaction measure,  and optionally penalize high CVaR in the utility.
 
 - Proposals with very long pickup distances are rejected with probability 0. Run MH under multiple (time threshold, CVaR threshold) configurations and compare results.

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def load_taxi_data(path, n_rows=100000, seed=0):
    """
    Load a subset of the taxi dataset.
    """
    rng = np.random.default_rng(seed)
    df = pd.read_csv(path, nrows=n_rows)

    # Clean durations: keep trips between 30 seconds and 2 hours
    df = df[(df["trip_duration"] > 30) & (df["trip_duration"] < 7200)]

    # Drop rows with missing coordinates
    df = df.dropna(subset=[
        "pickup_longitude", "pickup_latitude",
        "dropoff_longitude", "dropoff_latitude"
    ])

    df = df.reset_index(drop=True)
    return df



# I feel like this is a more accurate way to calculate distance but I feel like simple distance calculation is fine for this project also lol
def haversine(lon1, lat1, lon2, lat2):
    """
    Compute great-circle distance between two points (in km). Uses the haversine formula on a sphere.
    """
    R = 6371.0
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (
        np.sin(dlat / 2.0) ** 2
        + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    )
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c


In [ ]:
# Chnage this to be relative path later
taxi_path = "C:\\Users\\Ethan\\Desktop\\NYCTaxiData.csv"

df = load_taxi_data(taxi_path, n_rows=100000, seed=42)
df.head()


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,3/14/2016 17:24,3/14/2016 17:32,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,6/12/2016 0:43,6/12/2016 0:54,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,1/19/2016 11:35,1/19/2016 12:10,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,4/6/2016 19:32,4/6/2016 19:39,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,3/26/2016 13:30,3/26/2016 13:38,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [ ]:
dur = df["trip_duration"].to_numpy().astype(float)

# I'm not sure if we still need to filter durations since we want to focus on all trips
# dur = dur[(dur > 30) & (dur < 7200)]

log_dur = np.log(dur)
trip_mu_global = log_dur.mean()
trip_sigma_global = log_dur.std()

print(f"Global lognormal params for trip_duration:")
print(f"  mu    = {trip_mu_global:.3f}")
print(f"  sigma = {trip_sigma_global:.3f}")


Global lognormal params for trip_duration:
  mu    = 6.470
  sigma = 0.742


In [ ]:
def build_matching_scenario(df, n_riders=8, n_drivers=12, seed=0):
    """
    Build a small matching scenario from the taxi data.

    - Riders: random trips acting as customers needing a ride.
    - Drivers: random pickup locations acting as driver starting points.

    Returns a dictionary with coordinates and trip durations.
    """
    rng = np.random.default_rng(seed)

    # Sample riders
    rider_idx = rng.choice(len(df), size=n_riders, replace=False)
    riders = df.loc[rider_idx].reset_index(drop=True)

    # Sample drivers
    driver_idx = rng.choice(len(df), size=n_drivers, replace=False)
    drivers = df.loc[driver_idx].reset_index(drop=True)

    rider_pickup_lon = riders["pickup_longitude"].to_numpy()
    rider_pickup_lat = riders["pickup_latitude"].to_numpy()

    driver_lon = drivers["pickup_longitude"].to_numpy()
    driver_lat = drivers["pickup_latitude"].to_numpy()

    trip_durations = riders["trip_duration"].to_numpy().astype(float)

    scenario = {
        "riders": riders,
        "drivers": drivers,
        "rider_pickup_lon": rider_pickup_lon,
        "rider_pickup_lat": rider_pickup_lat,
        "driver_lon": driver_lon,
        "driver_lat": driver_lat,
        "trip_durations": trip_durations,
    }
    return scenario


scenario = build_matching_scenario(df, n_riders=8, n_drivers=12, seed=123)
scenario["riders"].head()


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2289553,2,3/28/2016 9:20,3/28/2016 9:32,6,-73.988342,40.731522,-74.003029,40.749149,N,713
1,id2180361,1,3/11/2016 18:20,3/11/2016 18:48,2,-73.993576,40.747108,-73.996223,40.711433,N,1683
2,id2195923,2,6/7/2016 12:40,6/7/2016 13:24,1,-73.959671,40.779652,-73.970421,40.754341,N,2606
3,id1890768,2,6/7/2016 21:48,6/7/2016 21:58,1,-73.965469,40.765938,-73.982124,40.771385,N,614
4,id3101055,2,2/7/2016 15:07,2/7/2016 15:11,6,-73.978096,40.741798,-73.987595,40.735756,N,230


In [ ]:
def lognormal_params_from_mean_cv(mean, cv):
    """Given desired mean and coefficient of variation (std/mean),
    return (mu, sigma) for the underlying Normal of a LogNormal.
    """
    # Use a CV bc it's easier to specify than stddev allowing scale-invariant variability
    variance = (cv * mean) ** 2
    sigma2 = np.log(1.0 + variance / (mean ** 2))
    sigma = np.sqrt(sigma2)
    mu = np.log(mean) - 0.5 * sigma2
    return mu, sigma


def sample_lognormal(mu, sigma, size, rng):
    """Sample from LogNormal(mu, sigma^2) where mu, sigma are Normal parameters."""
    return np.exp(rng.normal(mu, sigma, size=size))


def build_edge_time_parameters(
    scenario,
    trip_mu_global,
    trip_sigma_global,
    avg_speed_kmph=18.0,
    pickup_cv=0.2,
):
    """Build lognormal models for edge times (pickup + trip) between
    each rider and driver.

    For rider i and driver j:
      total time T_ij = pickup_ij + trip_i
    where pickup_ij and trip_i are both lognormals.

    - trip_i uses global lognormal parameters from the data.
    - pickup_ij has mean = distance(i,j) / avg_speed, with CV = pickup_cv.
    """
    rider_lon = scenario["rider_pickup_lon"]
    rider_lat = scenario["rider_pickup_lat"]
    driver_lon = scenario["driver_lon"]
    driver_lat = scenario["driver_lat"]

    n_riders = len(rider_lon)
    n_drivers = len(driver_lon)

    # Distance matrix (km) rider i -> driver j
    dist_km = np.zeros((n_riders, n_drivers))
    for i in range(n_riders):
        dist_km[i, :] = haversine(
            np.full(n_drivers, rider_lon[i]),
            np.full(n_drivers, rider_lat[i]),
            driver_lon,
            driver_lat,
        )

    # Mean pickup times (seconds)
    mean_pickup_sec = (dist_km / avg_speed_kmph) * 3600.0

    # Trip time lognormal params (same for all riders in this simple version)
    trip_mu = np.full(n_riders, trip_mu_global)
    trip_sigma = np.full(n_riders, trip_sigma_global)

    # Pickup time lognormal params for each (i, j)
    pickup_mu = np.zeros((n_riders, n_drivers))
    pickup_sigma = np.zeros((n_riders, n_drivers))

    for i in range(n_riders):
        for j in range(n_drivers):
            mean_ij = max(mean_pickup_sec[i, j], 10.0)  # at least 10s
            pickup_mu[i, j], pickup_sigma[i, j] = lognormal_params_from_mean_cv(
                mean_ij, pickup_cv
            )

    params = {
        "dist_km": dist_km,
        "mean_pickup_sec": mean_pickup_sec,
        "trip_mu": trip_mu,
        "trip_sigma": trip_sigma,
        "pickup_mu": pickup_mu,
        "pickup_sigma": pickup_sigma,
    }
    return params


params = build_edge_time_parameters(
    scenario,
    trip_mu_global=trip_mu_global,
    trip_sigma_global=trip_sigma_global,
    avg_speed_kmph=18.0,
    pickup_cv=0.2,
)

params["dist_km"]


array([[ 4.25966189,  4.59917563,  5.49901929,  1.64436676,  1.77315342,
        19.7130805 ,  5.73314536,  3.16289985,  6.07265421,  1.89188892,
         1.18447048,  3.46778009],
       [ 2.48794957,  3.45909978,  4.30580767,  1.47011132,  3.26120568,
        21.00833713,  4.51059539,  1.38111491,  4.46278141,  3.14259886,
         0.64903425,  4.89261395],
       [ 3.23540596,  1.29408001,  0.43225979,  4.27631448,  7.62839584,
        21.15776108,  0.19752579,  3.76364514,  1.74510266,  7.63657473,
         5.15321438,  9.32380824],
       [ 2.57059348,  0.31485165,  1.21457763,  2.67665892,  6.04990611,
        20.43840181,  1.45137878,  2.63210086,  2.46399437,  6.08224257,
         3.64490846,  7.74677435],
       [ 3.40934363,  3.19959103,  4.09962055,  0.21338492,  3.2034454 ,
        19.59778348,  4.33845436,  2.45017039,  4.8771224 ,  3.30790231,
         1.33947334,  4.8944472 ],
       [ 3.61999641,  4.72447128,  5.58680149,  2.20959174,  2.12201557,
        21.00334649,  

In [ ]:
def simulate_matching_times(
    matching,
    params,
    time_threshold,
    n_mc=2000,
    alpha=0.95,
    rng=None,
):
    """
    Given a matching, simulate total times and compute:
      - Utility = log P(max_i T_i <= time_threshold)
      - CVaR_alpha of the mean rider time
    using Monte Carlo.
    """
    if rng is None:
        rng = np.random.default_rng()

    trip_mu = params["trip_mu"]
    trip_sigma = params["trip_sigma"]
    pickup_mu = params["pickup_mu"]
    pickup_sigma = params["pickup_sigma"]

    n_riders = len(matching)

    max_totals = np.zeros(n_mc)
    mean_totals = np.zeros(n_mc)

    for s in range(n_mc):
        totals = np.zeros(n_riders)
        for i in range(n_riders):
            j = matching[i]
            # sample pickup and trip times
            pickup_ij = sample_lognormal(
                pickup_mu[i, j], pickup_sigma[i, j], size=1, rng=rng
            )[0]
            trip_i = sample_lognormal(
                trip_mu[i], trip_sigma[i], size=1, rng=rng
            )[0]
            totals[i] = pickup_ij + trip_i

        max_totals[s] = totals.max()
        mean_totals[s] = totals.mean()

    # Utility
    prob = np.mean(max_totals <= time_threshold)
    eps = 1e-12
    log_prob = np.log(prob + eps)

    # CVaR of mean times at level alpha
    q_alpha = np.quantile(mean_totals, alpha)
    tail = mean_totals[mean_totals >= q_alpha]
    if len(tail) == 0:
        cvar = q_alpha
    else:
        cvar = tail.mean()

    return log_prob, cvar


# Ensure function works on a simple matching
rng = np.random.default_rng(0)
n_riders = len(scenario["riders"])
n_drivers = len(scenario["drivers"])

matching0 = np.arange(n_riders)
log_u0, cvar0 = simulate_matching_times(
    matching0, params, time_threshold=1800, n_mc=500, rng=rng
)
print("Sample matching log utility:", log_u0)
print("Sample matching CVaR:", cvar0)


Sample matching log utility: -27.631021115928547
Sample matching CVaR: 2525.8925529353432


In [ ]:
def random_initial_matching(n_riders, n_drivers, rng=None):
    """Provide a random injective matching making sure each rider gets a unique driver."""
    if rng is None:
        rng = np.random.default_rng()
    drivers = np.arange(n_drivers)
    rng.shuffle(drivers)
    return drivers[:n_riders].copy()


def propose_matching_swap(matching, rng=None):
    """
    Symmetric proposal to pick two riders and swap their drivers.
    """
    if rng is None:
        rng = np.random.default_rng()
    new_match = matching.copy()
    i, j = rng.choice(len(matching), size=2, replace=False)
    new_match[i], new_match[j] = new_match[j], new_match[i]
    return new_match


def metropolis_hastings(
    params,
    max_dist_km=None,
    time_threshold=1800,
    cvar_threshold=None,
    # This implements the soft penalty (subtract λ * (CVaR - threshold) from log_prob) on CVaR but we can also do the hard cut off
    cvar_penalty_weight=0.0,
    alpha=0.95,
    n_iters=2000,
    burn_in=500,
    n_mc_utility=1000,
    seed=0,
    hard_cvar_cutoff=False,    # if True: CVaR > threshold -> utility = -inf
):
    """
    Run Metropolis-Hastings over matchings.

    Target:
      π(M) ∝ exp(U(M)), where U(M) is utility:

        U(M) = log P(max_i T_i <= time_threshold)
               - lambda * max(0, CVaR(M) - cvar_threshold)

    Distance constraint:
      - If max_dist_km is not None and any assigned edge exceeds it,
        proposals are rejected immediately (transition prob 0).
    """
    rng = np.random.default_rng(seed)

    dist_km = params["dist_km"]
    n_riders, n_drivers = dist_km.shape

    def feasible_by_distance(m):
        if max_dist_km is None:
            return True
        d = dist_km[np.arange(n_riders), m]
        return np.all(d <= max_dist_km)

    # Initial matching
    current = random_initial_matching(n_riders, n_drivers, rng)

    # Try to find an initial feasible matching wrt distance
    if max_dist_km is not None and not feasible_by_distance(current):
        found = False
        for _ in range(2000):
            cand = random_initial_matching(n_riders, n_drivers, rng)
            if feasible_by_distance(cand):
                current = cand
                found = True
                break
        if not found:
            print(
                "Could not find an initial matching satisfying "
                f"max_dist_km={max_dist_km}. Need to disable distance constraint."
            )

    def apply_cvar_to_utility(log_prob, cvar):
        if cvar_threshold is None:
            return log_prob
        if hard_cvar_cutoff and (cvar > cvar_threshold):
            return -np.inf
        if cvar_penalty_weight > 0.0:
            excess = max(0.0, cvar - cvar_threshold)
            return log_prob - cvar_penalty_weight * excess
        return log_prob

    # Evaluate initial state
    current_log_prob, current_cvar = simulate_matching_times(
        current, params, time_threshold,
        n_mc=n_mc_utility, alpha=alpha, rng=rng,
    )
    current_U = apply_cvar_to_utility(current_log_prob, current_cvar)

    samples = []
    log_utils = []
    cvars = []

    for it in range(n_iters):
        proposal = propose_matching_swap(current, rng)

        # Distance-based hard constraint
        if max_dist_km is not None and not feasible_by_distance(proposal):
            accept = False
        else:
            prop_log_prob, prop_cvar = simulate_matching_times(
                proposal, params, time_threshold,
                n_mc=n_mc_utility, alpha=alpha, rng=rng,
            )
            prop_U = apply_cvar_to_utility(prop_log_prob, prop_cvar)

            # MH acceptance ratio
            log_ratio = prop_U - current_U
            if np.isneginf(prop_U) and np.isneginf(current_U):
                accept = False
            elif log_ratio >= 0:
                accept = True
            else:
                u = rng.uniform()
                accept = (np.log(u) < log_ratio)

        if accept:
            current = proposal
            current_log_prob, current_cvar = prop_log_prob, prop_cvar
            current_U = apply_cvar_to_utility(current_log_prob, current_cvar)

        if it >= burn_in:
            samples.append(current.copy())
            log_utils.append(current_log_prob)
            cvars.append(current_cvar)

    samples = np.array(samples)
    log_utils = np.array(log_utils)
    cvars = np.array(cvars)

    return samples, log_utils, cvars


In [ ]:
# Some example configurations that chat came up with
configs = [
    {"time_threshold": 1200, "cvar_threshold": 1500},  # 20 min
    {"time_threshold": 1800, "cvar_threshold": 2000},  # 30 min
    {"time_threshold": 2400, "cvar_threshold": 2500},  # 40 min
]

results = []

for cfg in configs:
    print("\n=======================================")
    print(f"Config: time_threshold={cfg['time_threshold']} sec, "
          f"cvar_threshold={cfg['cvar_threshold']} sec")
    print("=======================================")

    samples, log_utils, cvars = metropolis_hastings(
        params,
        max_dist_km=10.0,                    # very long pickup distance
        time_threshold=cfg["time_threshold"],
        cvar_threshold=cfg["cvar_threshold"],
        cvar_penalty_weight=0.001,           # soft CVaR penalty
        alpha=0.95,
        n_iters=800,                         # increase for more serious runs
        burn_in=200,
        n_mc_utility=500,
        seed=0,
        hard_cvar_cutoff=False,              # set True to hard-reject high CVaR
    )

    print(f"Collected {len(samples)} post–burn-in samples")
    print(f"Mean log utility (log P(max <= T)): {np.mean(log_utils):.3f}")
    print(f"Mean CVaR (mean time, α=0.95):      {np.mean(cvars):.1f} sec")

    if len(cvars) > 0:
        q = np.quantile(cvars, [0.5, 0.8, 0.9, 0.95])
        print("CVaR quantiles (50%, 80%, 90%, 95%):", q)

    results.append((cfg, samples, log_utils, cvars))



Config: time_threshold=1200 sec, cvar_threshold=1500 sec
Collected 600 post–burn-in samples
Mean log utility (log P(max <= T)): -4.942
Mean CVaR (mean time, α=0.95):      1949.1 sec
CVaR quantiles (50%, 80%, 90%, 95%): [1935.30474884 2021.40736361 2078.43827637 2157.56572632]

Config: time_threshold=1800 sec, cvar_threshold=2000 sec
Collected 600 post–burn-in samples
Mean log utility (log P(max <= T)): -2.160
Mean CVaR (mean time, α=0.95):      2075.6 sec
CVaR quantiles (50%, 80%, 90%, 95%): [2064.96491143 2174.76909008 2237.31968832 2287.76120057]

Config: time_threshold=2400 sec, cvar_threshold=2500 sec
Collected 600 post–burn-in samples
Mean log utility (log P(max <= T)): -0.963
Mean CVaR (mean time, α=0.95):      2151.2 sec
CVaR quantiles (50%, 80%, 90%, 95%): [2147.16673644 2265.12579898 2322.84178368 2371.56418854]
